<a href="https://colab.research.google.com/github/ryderwishart/biblical-machine-learning/blob/main/gpt-insights/macula_bible_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Empowering Bots with Data

This notebook exemplifies how to build a powerful question-answering system using OpenAI's Language Model (LLM), augmented with a specific knowledge database—in this case, the Bible. The aim of this combined system is to provide accurate responses to queries related to the Bible, as well as to demonstrate the capabilities of LLMs when combined with structured data sources, thus empowering them with long-term memory and specificity.

The main components of this notebook are:

1. **Building a Bible QA System**: A question-answering system specifically trained on the Bible is created. This involves vectorizing the Bible text and storing it in a Vectorstore database. The Bible QA system is built using the RetrievalQA method from the langchain.chains module, which leverages the power of the LLM to reason about and retrieve information from the Bible database.

2. **Initializing an LLM Agent**: An LLM Agent is initialized with the Bible QA system as a tool. This allows the agent to utilize the Bible QA system when answering questions, essentially enabling it to "remember" information from the Bible and reason about it. The LLM Agent acts as a router and decision-maker, determining how and when to use the Bible QA system based on the input question.

3. **Running Queries**: The combined system is then used to answer complex, multi-step questions related to the Bible. The LLM Agent's ability to perform multi-step reasoning is showcased, with the agent using the Bible QA system to answer individual parts of the question and then combining those answers into a final response.

This notebook thus exemplifies the use of LLMs as a reasoning tool, enhanced by a specific knowledge database. This approach not only improves the accuracy of responses to specific queries but also provides the LLM with a form of long-term memory, allowing it to consistently access and reason about a large, fixed body of knowledge.

In [1]:
!pip install langchain openai chromadb tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
import os
os.environ["LANGCHAIN_TRACING"] = "false"

# Provision data for context

In [2]:
import pandas as pd

if "macula-greek.tsv" not in [path for path in os.listdir()]:
    !wget - O macula-greek.tsv https://raw.githubusercontent.com/Clear-Bible/macula-greek/main/Nestle1904/TSV/macula-greek.tsv
tsv_file = "macula-greek.tsv"
data = pd.read_csv(tsv_file, sep="\t")
data.head()

,xml:id,ref,role,class,type,gloss,text,after,lemma,normalized,...,case,tense,voice,mood,degree,domain,ln,frame,subjref,referent
0,n40001001001,MAT 1:1!1,NaN,noun,common,[The] book,Βίβλος,,βίβλος,Βίβλος,...,nominative,NaN,NaN,NaN,NaN,033005,33.38,NaN,NaN,NaN
1,n40001001002,MAT 1:1!2,NaN,noun,common,of [the] genealogy,γενέσεως,,γένεσις,γενέσεως,...,genitive,NaN,NaN,NaN,NaN,010002 033003,10.24 33.19,NaN,NaN,NaN
2,n40001001003,MAT 1:1!3,NaN,noun,proper,of Jesus,Ἰησοῦ,,Ἰησοῦς,Ἰησοῦ,...,genitive,NaN,NaN,NaN,NaN,093001,93.169a,NaN,NaN,NaN
3,n40001001004,MAT 1:1!4,NaN,noun,proper,Christ,Χριστοῦ,,Χριστός,Χριστοῦ,...,genitive,NaN,NaN,NaN,NaN,093001,93.387,NaN,NaN,NaN
4,n40001001005,MAT 1:1!5,NaN,noun,common,son,υἱοῦ,,υἱός,υἱοῦ,...,genitive,NaN,NaN,NaN,NaN,010002,10.30,NaN,NaN,NaN


In [3]:
# Extract book, chapter, and verse into separate columns
data[['book', 'chapter', 'verse']] = data['ref'].str.extract(r'(\d?[A-Z]+)\s(\d+):(\d+)')

# Add columns for book + chapter, and book + chapter + verse for easier grouping
data['book_chapter'] = data['book'] + ' ' + data['chapter'].astype(str)
data['book_chapter_verse'] = data['book_chapter'] + ':' + data['verse'].astype(str)

# Display the updated data frame
data.head()

,xml:id,ref,role,class,type,gloss,text,after,lemma,normalized,...,domain,ln,frame,subjref,referent,book,chapter,verse,book_chapter,book_chapter_verse
0,n40001001001,MAT 1:1!1,NaN,noun,common,[The] book,Βίβλος,,βίβλος,Βίβλος,...,033005,33.38,NaN,NaN,NaN,MAT,1,1,MAT 1,MAT 1:1
1,n40001001002,MAT 1:1!2,NaN,noun,common,of [the] genealogy,γενέσεως,,γένεσις,γενέσεως,...,010002 033003,10.24 33.19,NaN,NaN,NaN,MAT,1,1,MAT 1,MAT 1:1
2,n40001001003,MAT 1:1!3,NaN,noun,proper,of Jesus,Ἰησοῦ,,Ἰησοῦς,Ἰησοῦ,...,093001,93.169a,NaN,NaN,NaN,MAT,1,1,MAT 1,MAT 1:1
3,n40001001004,MAT 1:1!4,NaN,noun,proper,Christ,Χριστοῦ,,Χριστός,Χριστοῦ,...,093001,93.387,NaN,NaN,NaN,MAT,1,1,MAT 1,MAT 1:1
4,n40001001005,MAT 1:1!5,NaN,noun,common,son,υἱοῦ,,υἱός,υἱοῦ,...,010002,10.30,NaN,NaN,NaN,MAT,1,1,MAT 1,MAT 1:1


In [4]:
for column in data.columns:
    print(column)

xml:id
ref
role
class
type
gloss
text
after
lemma
normalized
strong
morph
person
number
gender
case
tense
voice
mood
degree
domain
ln
frame
subjref
referent
book
chapter
verse
book_chapter
book_chapter_verse


In [77]:
import numpy as np

# Initialize lists
text_list = []
dict_list = []
id_list = []
glosses_list = []

# Group the DataFrame by 'book_chapter_verse'
grouped = data.groupby('book_chapter_verse')

for name, group in grouped:
    # Combine the 'text' and 'after' fields into a single string for each group
    text = ''.join(group['text'] + group['after'].replace(np.nan, '', regex=True) + ' ')
    text_list.append(text)

    # Extract book, chapter, and verse from the group
    book = group['book'].values[0]
    chapter = group['book_chapter'].str.split().str[1].values[0]
    verse = group['book_chapter_verse'].str.split(':').str[1].values[0]
    b_c_v = group['book_chapter_verse'].values[0]
    
    # Use the 'xml:id' field to create a list of IDs for the verse (joined by pipes)
    id_entry = '|'.join(group['xml:id'].tolist())
    
    # ... Same for glosses
    verse_gloss = ''.join(group['gloss'].replace(np.nan, '[no gloss]', regex=True) + group['after'].replace(np.nan, '', regex=True) + ' ')
    glosses_list.append(b_c_v + ' - ' + verse_gloss)
    
    # Add metadata for verse
    dict_entry = {'source': b_c_v, 'book': book, 'chapter': chapter, 'verse': verse, 'ids': id_entry, 'gloss': verse_gloss}
    dict_list.append(dict_entry)
    
    id_list.append(b_c_v)

# Print the lists for testing
print(text_list[:5])
print(dict_list[:5])
print(id_list[:5])

['Οὐ  θέλω  γὰρ  ὑμᾶς  ἀγνοεῖν, ἀδελφοί, ὅτι  οἱ  πατέρες  ἡμῶν  πάντες  ὑπὸ  τὴν  νεφέλην  ἦσαν  καὶ  πάντες  διὰ  τῆς  θαλάσσης  διῆλθον, ', 'μηδὲ  γογγύζετε, καθάπερ  τινὲς  αὐτῶν  ἐγόγγυσαν, καὶ  ἀπώλοντο  ὑπὸ  τοῦ  ὀλοθρευτοῦ. ', 'ταῦτα  δὲ  τυπικῶς  συνέβαινεν  ἐκείνοις, ἐγράφη  δὲ  πρὸς  νουθεσίαν  ἡμῶν, εἰς  οὓς  τὰ  τέλη  τῶν  αἰώνων  κατήντηκεν. ', 'Ὥστε  ὁ  δοκῶν  ἑστάναι  βλεπέτω  μὴ  πέσῃ, ', 'πειρασμὸς  ὑμᾶς  οὐκ  εἴληφεν  εἰ  μὴ  ἀνθρώπινος· πιστὸς  δὲ  ὁ  Θεός, ὃς  οὐκ  ἐάσει  ὑμᾶς  πειρασθῆναι  ὑπὲρ  ὃ  δύνασθε, ἀλλὰ  ποιήσει  σὺν  τῷ  πειρασμῷ  καὶ  τὴν  ἔκβασιν  τοῦ  δύνασθαι  ὑπενεγκεῖν. ']
[{'source': '1CO 10:1', 'book': '1CO', 'chapter': '10', 'verse': '1', 'ids': 'n46010001001|n46010001002|n46010001003|n46010001004|n46010001005|n46010001006|n46010001007|n46010001008|n46010001009|n46010001010|n46010001011|n46010001012|n46010001013|n46010001014|n46010001015|n46010001016|n46010001017|n46010001018|n46010001019|n46010001020|n46010001021', 'gloss': 'Not  I want  for  y

In [6]:
import getpass
secret_key = getpass.getpass('Enter OpenAI secret key: ')
os.environ['OPENAI_API_KEY'] = secret_key

Enter OpenAI secret key: ··········


In [69]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.document_loaders import TextLoader
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from pathlib import Path

# Load Language Model
llm = OpenAI(temperature=0)

In [8]:
# # Load and process the Bible text
# doc_path = filename
# loader = TextLoader(doc_path)
# documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# texts = text_splitter.split_documents(documents)

In [78]:
# Create embeddings and store in a vectorstore
embeddings = OpenAIEmbeddings()
collection = Chroma("bible-qa", embeddings)
"""
collection_name: str = _LANGCHAIN_DEFAULT_COLLECTION_NAME,
        embedding_function: Optional[Embeddings] = None,
        persist_directory: Optional[str] = None,
        client_settings: Optional[chromadb.config.Settings] = None,
        collection_metadata: Optional[Dict] = None,
        client: Optional[chromadb.Client] = None,
"""



# bible_chroma = Chroma.from_documents(texts, embeddings, collection_name="kjv-bible")

'\ncollection_name: str = _LANGCHAIN_DEFAULT_COLLECTION_NAME,\n        embedding_function: Optional[Embeddings] = None,\n        persist_directory: Optional[str] = None,\n        client_settings: Optional[chromadb.config.Settings] = None,\n        collection_metadata: Optional[Dict] = None,\n        client: Optional[chromadb.Client] = None,\n'

In [10]:
# Add greek texts with metadata
collection.add_texts(
    texts=text_list,
    metadatas=dict_list,
    ids=id_list
    )

['1CO 10:1',
 '1CO 10:10',
 '1CO 10:11',
 '1CO 10:12',
 '1CO 10:13',
 '1CO 10:14',
 '1CO 10:15',
 '1CO 10:16',
 '1CO 10:17',
 '1CO 10:18',
 '1CO 10:19',
 '1CO 10:2',
 '1CO 10:20',
 '1CO 10:21',
 '1CO 10:22',
 '1CO 10:23',
 '1CO 10:24',
 '1CO 10:25',
 '1CO 10:26',
 '1CO 10:27',
 '1CO 10:28',
 '1CO 10:29',
 '1CO 10:3',
 '1CO 10:30',
 '1CO 10:31',
 '1CO 10:32',
 '1CO 10:33',
 '1CO 10:4',
 '1CO 10:5',
 '1CO 10:6',
 '1CO 10:7',
 '1CO 10:8',
 '1CO 10:9',
 '1CO 11:1',
 '1CO 11:10',
 '1CO 11:11',
 '1CO 11:12',
 '1CO 11:13',
 '1CO 11:14',
 '1CO 11:15',
 '1CO 11:16',
 '1CO 11:17',
 '1CO 11:18',
 '1CO 11:19',
 '1CO 11:2',
 '1CO 11:20',
 '1CO 11:21',
 '1CO 11:22',
 '1CO 11:23',
 '1CO 11:24',
 '1CO 11:25',
 '1CO 11:26',
 '1CO 11:27',
 '1CO 11:28',
 '1CO 11:29',
 '1CO 11:3',
 '1CO 11:30',
 '1CO 11:31',
 '1CO 11:32',
 '1CO 11:33',
 '1CO 11:34',
 '1CO 11:4',
 '1CO 11:5',
 '1CO 11:6',
 '1CO 11:7',
 '1CO 11:8',
 '1CO 11:9',
 '1CO 12:1',
 '1CO 12:10',
 '1CO 12:11',
 '1CO 12:12',
 '1CO 12:13',
 '1CO 12:14

In [79]:
# Add glossed texts with metadata
collection.add_texts(
    texts=glosses_list,
    metadatas=dict_list,
    ids=[id + '_gloss' for id in id_list]
    )

['1CO 10:1_gloss',
 '1CO 10:10_gloss',
 '1CO 10:11_gloss',
 '1CO 10:12_gloss',
 '1CO 10:13_gloss',
 '1CO 10:14_gloss',
 '1CO 10:15_gloss',
 '1CO 10:16_gloss',
 '1CO 10:17_gloss',
 '1CO 10:18_gloss',
 '1CO 10:19_gloss',
 '1CO 10:2_gloss',
 '1CO 10:20_gloss',
 '1CO 10:21_gloss',
 '1CO 10:22_gloss',
 '1CO 10:23_gloss',
 '1CO 10:24_gloss',
 '1CO 10:25_gloss',
 '1CO 10:26_gloss',
 '1CO 10:27_gloss',
 '1CO 10:28_gloss',
 '1CO 10:29_gloss',
 '1CO 10:3_gloss',
 '1CO 10:30_gloss',
 '1CO 10:31_gloss',
 '1CO 10:32_gloss',
 '1CO 10:33_gloss',
 '1CO 10:4_gloss',
 '1CO 10:5_gloss',
 '1CO 10:6_gloss',
 '1CO 10:7_gloss',
 '1CO 10:8_gloss',
 '1CO 10:9_gloss',
 '1CO 11:1_gloss',
 '1CO 11:10_gloss',
 '1CO 11:11_gloss',
 '1CO 11:12_gloss',
 '1CO 11:13_gloss',
 '1CO 11:14_gloss',
 '1CO 11:15_gloss',
 '1CO 11:16_gloss',
 '1CO 11:17_gloss',
 '1CO 11:18_gloss',
 '1CO 11:19_gloss',
 '1CO 11:2_gloss',
 '1CO 11:20_gloss',
 '1CO 11:21_gloss',
 '1CO 11:22_gloss',
 '1CO 11:23_gloss',
 '1CO 11:24_gloss',
 '1CO 11:25

In [80]:
# Inspect some texts 
print('MAT 1:1 -->', collection.search('MAT 1:1', search_type='similarity'))
print('blind -->', collection.search('blind', search_type='similarity'))


MAT 1:1 --> [Document(page_content='MAT 1:1 - [The] book  of [the] genealogy  of Jesus  Christ  son  of David  son  of Abraham. ', metadata={'source': 'MAT 1:1', 'book': 'MAT', 'chapter': '1', 'verse': '1', 'ids': 'n40001001001|n40001001002|n40001001003|n40001001004|n40001001005|n40001001006|n40001001007|n40001001008', 'gloss': '[The] book  of [the] genealogy  of Jesus  Christ  son  of David  son  of Abraham. '}), Document(page_content='MAT 1:25 - and  not  knew  her  until  that  she had brought forth  a son· and  he called  the  name  of Him  Jesus. ', metadata={'source': 'MAT 1:25', 'book': 'MAT', 'chapter': '1', 'verse': '25', 'ids': 'n40001025001|n40001025002|n40001025003|n40001025004|n40001025005|n40001025006|n40001025007|n40001025008|n40001025009|n40001025010|n40001025011|n40001025012|n40001025013|n40001025014', 'gloss': 'and  not  knew  her  until  that  she had brought forth  a son· and  he called  the  name  of Him  Jesus. '}), Document(page_content='MAT 1:21 - She will bear 

# Chains

In [73]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [81]:
docs = collection.search('blind', search_type='similarity')
docs

[Document(page_content='2PE 1:9 - In whomever  for  not  are present  these things, blind  he is  being short sighted, forgetfulness  having received  of the  purification  the  former  of him  sins. ', metadata={'source': '2PE 1:9', 'book': '2PE', 'chapter': '1', 'verse': '9', 'ids': 'n61001009001|n61001009002|n61001009003|n61001009004|n61001009005|n61001009006|n61001009007|n61001009008|n61001009009|n61001009010|n61001009011|n61001009012|n61001009013|n61001009014|n61001009015|n61001009016', 'gloss': 'In whomever  for  not  are present  these things, blind  he is  being short sighted, forgetfulness  having received  of the  purification  the  former  of him  sins. '}),
 Document(page_content='JHN 9:1 - And  passing by  He saw  a man  blind  from  birth. ', metadata={'source': 'JHN 9:1', 'book': 'JHN', 'chapter': '9', 'verse': '1', 'ids': 'n43009001001|n43009001002|n43009001003|n43009001004|n43009001005|n43009001006|n43009001007', 'gloss': 'And  passing by  He saw  a man  blind  from  b

In [105]:
from langchain.retrievers.tfidf import TFIDFRetriever

class ExtendedTFIDFRetriever(TFIDFRetriever):
    def get_distinctive_terms(self, doc_index, top_n=10):
        feature_names = self.vectorizer.get_feature_names_out()
        tfidf_vector = self.tfidf_array[doc_index]

        # Convert the sparse matrix row to a dense array
        tfidf_array_dense = tfidf_vector.toarray().flatten()

        # Get indices of top_n features
        top_indices = tfidf_array_dense.argsort()[-top_n:][::-1]

        # Get the corresponding feature names
        top_terms = [feature_names[i] for i in top_indices]

        # TODO: Filter out stopwords using list generated in one of the missional-ai notebooks

        return top_terms

texts = [doc.page_content for doc in docs] # get page content out of the queried docs

retriever = ExtendedTFIDFRetriever.from_texts(texts)
distinctive_terms = retriever.get_distinctive_terms(0) # index of document in texts
print(distinctive_terms)

['of', 'the', 'present', 'received', 'being', 'short', 'sighted', 'sins', 'for', 'purification']


In [99]:


def get_similar_docs(query_string):
    return collection.search(query_string, search_type='similarity')

def get_range_of_examples(query_string):
    return collection.search(query_string, search_type='mmr')

def get_prosaic_context_for_verse(verse_ref_string):
    return 'This is some prosaic context placeholder content.'

# prompt should be something like this:
"""
(
    "You are a language model trained in linguistics," 
    " and you are great at summarizing structured data while"
    " focusing on linguistic features without delving into theological issues."
    " Please provide a concise textual commentary on the given {linguistic_data}"
    " (use the treedown representation for the larger text context of the target word)"
    " by examining its key lexical choices, syntactic structures, discourse organization,"
    " social context, and cultural references. Avoid personal opinions and maintain objectivity."
    " Illuminate the passage's nuances, foster clarity, and establish connections within the work,"
    " empowering readers to grasp the author's intentions and the interplay between language and content."
    " Please format your output using the following headings:"
    " 1. Lexical features"
    " 2. Syntactic context and function"
    " 3. Discourse context"
    " 4. Social context"
    " 5. Cultural/encyclopedic knowledge"    
).format(linguistic_data=linguistic_data)
"""

def answer_question_with_context(input_question):
    # first, find the relevant bible verse source ids
    # second, get the prosaic context for the first verse (later this can be a multi-verse thing...)

In [82]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "Is blindness a generally positive or negative trait?"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' Blindness is generally seen as a negative trait in the Bible.\nSOURCES: 2PE 1:9, JHN 9:1, MAT 15:14, JHN 9:40'}

In [98]:
# # Create a RetrievalQA tool with this vectorstore
# bible_tool_no_sources = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=collection.as_retriever())

# tools = [
#     Tool(
#         name = "Bible QA System",
#         func=bible_tool_no_sources.run,
#         description="useful for when you need to find relevant documents for answering the user's questions. Input should be a fully formed question.",
#         return_direct=True,  # If you want to use the agent as a router and return results directly
#     ),
# ]

# Create a RetrievalQAWithSourcesChain tool with this vectorstore
# bible_tool = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, chain_type="stuff", retriever=collection.as_retriever())

# tools = [
#     Tool(
#         name = "Bible QA System",
#         func=bible_tool.run,
#         description="useful for when you need to answer questions about the Bible. Input should be a fully formed question.",
#         # return_direct=True  # If you want to use the agent as a router and return results directly
#     ),
# ]

# Create a RetrievalQAWithSourcesChain tool with this vectorstore
bible_doc_qa_chain = load_qa_with_sources_chain(llm=llm, chain_type="stuff", verbose=True)
query = "Is blindness a generally positive or negative trait?"
print('test run', bible_doc_qa_chain({"input_documents": docs, "question": query}, return_only_outputs=True))

tools = [
    Tool(
        name = "Bible QA System",
        func=bible_doc_qa_chain,
        description="useful for when you need to answer questions about the Bible. Input should be a fully formed question.",
        verbose=True
        # return_direct=True  # If you want to use the agent as a router and return results directly
    ),
]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.

11.7 Severability. The invalidity,


> Finished chain.

> Finished chain.
test run {'output_text': ' Blindness is generally seen as a negative trait in the Bible.\nSOURCES: 2PE 1:9, JHN 9:1, MAT 15:14, JHN 9:40'}


# Agents

In [94]:
# Initialize the agent
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [95]:
agent.run("How does Jesus heal a man born blind?")



> Entering new AgentExecutor chain...


 I need to find out what the Bible says about this
Action: Bible QA System
Action Input: How does Jesus heal a man born blind?

AttributeError: ignored

Here is a two-step question. The zero-shot react description Cannot handle the second part properly.

In [22]:
# agent.run("Where was Job from? Does that place get mentioned anywhere else in the Bible?")
agent.run("What is an ephod? Who wears one?")





> Entering new AgentExecutor chain...
 I need to find out what an ephod is and who wears one.
Action: Bible QA System
Action Input: What is an ephod? Who wears one?


Observation:  An ephod is a type of garment worn by priests in the Bible.


> Finished chain.


' An ephod is a type of garment worn by priests in the Bible.'

However, the self-ask with search agent can.

In [23]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

# Assuming you have already initialized your Bible QA system
bible_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=collection.as_retriever())

tools = [
    Tool(
        name="Intermediate Answer",
        func=bible_qa.run,
        description="useful for when you need to answer questions about the Bible. Input should be a fully formed question."
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)
# self_ask_with_search.run("Where was Job from? Does that place get mentioned in the Bible?")
self_ask_with_search.run("What is an ephod? Who wears one?")




> Entering new AgentExecutor chain...
 Yes.
Follow up: What is an ephod?


Intermediate answer:  I don't know.
Follow up: Where can I find information about an ephod?
Intermediate answer:  I don't know.
Follow up: Where can I find information about an ephod?
Intermediate answer:  I don't know.
Follow up: What is the definition of an ephod?
Intermediate answer:  I don't know.
Follow up: What is the definition of an ephod?
Intermediate answer:  I don't know.


KeyboardInterrupt: ignored

Much better.

## Next steps:

- Combine the self-ask agent with the Bible vector retrieval agent.
- Think through some custom tooling for exciting new applications such as:
 - Bible translation functionality in low- or no-resource languages
 - Extracting new structured datasets using text plus existing structured data that can be retrieved on the fly
 - Evaluating Bible translations using qualitative "metrics" (as opposed to more conventional ones) on the basis of structured data extracted from the back translation

Questions? Ideas? Reach out to ryderwishart (at) gmail dot com!